In [248]:
import os
import numpy as np
import csv
import tifffile as tiff  # Para leer archivos TIFF
from skimage.io import imread
from skimage.filters import sobel
from skimage.feature import hog
from tqdm import tqdm

# Directorios de las imágenes y las máscaras
directorio_satelital = "roads/sat"
directorio_gt = "roads/gt"

# Emparejar las imágenes con sus máscaras (suponiendo que tienen el mismo nombre)
pares_imagenes_mascaras = list(zip(sorted(os.listdir(directorio_satelital)), sorted(os.listdir(directorio_gt))))
print(f"Se encontraron {len(pares_imagenes_mascaras)} pares de imágenes y máscaras.")

# Nombre del archivo CSV
csv_file = "caracteristicas_imagenes_pixel_a_pixel.csv"

# Función para calcular HOG de toda la imagen
def calcular_hog(imagen):
    hog_features, _ = hog(imagen, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return hog_features

# Abrir el archivo CSV en modo escritura
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Escribir los encabezados del CSV
    writer.writerow(['R', 'G', 'B', 'sobel', 'hog', 'etiqueta'])
    
    # Cargar las imágenes y máscaras
    for imagen_idx, (imagen_nombre, mascara_nombre) in tqdm(enumerate(pares_imagenes_mascaras), desc="Cargando imágenes y máscaras"):
        # Leer la imagen satelital
        ruta_imagen = os.path.join(directorio_satelital, imagen_nombre)
        imagen = imread(ruta_imagen)
        
        # Leer la máscara de ground truth (con tifffile para imágenes TIFF)
        ruta_mascara = os.path.join(directorio_gt, mascara_nombre)
        mascara = tiff.imread(ruta_mascara)
        
        # Asegurarse de que la máscara tenga valores 0 y 255
        if np.max(mascara) <= 1:  # Si los valores están en el rango [0, 1]
            mascara = (mascara * 255).astype(np.uint8)

        # Identificar los píxeles de carretera y fondo
        indices_carretera = np.where(mascara == 255)  # Carretera
        indices_fondo = np.where(mascara == 0)      # Fondo

        # Número de píxeles de fondo que queremos seleccionar
        num_carretera = len(indices_carretera[0])
        num_fondo = len(indices_fondo[0])

        # Si los píxeles de fondo son más que los de carretera, hacer downsampling
        if num_fondo > num_carretera:
            downsampling_factor = num_fondo // num_carretera  # Ajustar el factor de downsampling
            indices_fondo_downsampled = indices_fondo[0][::downsampling_factor]  # Reducir los píxeles de fondo
        else:
            indices_fondo_downsampled = indices_fondo[0]  # Mantener todos los píxeles de fondo si son menores o iguales a los de carretera

        # Ahora aseguramos que los índices balanceados respeten las clases
        indices_balanceados = np.concatenate(( 
            # Índices de carretera con etiqueta 255
            np.array([255] * len(indices_carretera[0])),  
            # Índices de fondo balanceados con etiqueta 0
            np.array([0] * len(indices_fondo_downsampled)) 
        ))

        # Calcular Sobel y HOG para la imagen completa (en lugar de píxel por píxel)
        sobel_imagen = sobel(imagen[:, :, 0])  # Sobel sobre el canal rojo (puedes elegir otro canal también)
        hog_features = calcular_hog(imagen[:, :, 0])

        # Extraer características solo para los índices balanceados
        coordenadas = []
        caracteristicas_pixel = []

        for idx, etiqueta in zip(np.concatenate((indices_carretera[0], indices_fondo_downsampled)), indices_balanceados):
            i, j = divmod(idx, imagen.shape[1])  # Obtener las coordenadas (i, j) del índice lineal

            # Extraer los valores de los canales R, G, B
            r_value = imagen[i, j, 0]
            g_value = imagen[i, j, 1]
            b_value = imagen[i, j, 2]
            
            # Obtener el valor de Sobel y HOG para este píxel
            sobel_value = sobel_imagen[i, j]
            hog_value = hog_features[(i // 8) * (imagen.shape[1] // 8) + (j // 8)]  # Aproximación al índice de HOG

            # Guardar las características de este píxel
            caracteristicas_pixel.append([r_value, g_value, b_value, sobel_value, hog_value])
            coordenadas.append((i, j))  # Guardamos las coordenadas (i, j)

        # Escribir las características y etiquetas en el CSV
        for caracteristicas, etiqueta in zip(caracteristicas_pixel, indices_balanceados):
            writer.writerow(caracteristicas + [etiqueta])

Se encontraron 20 pares de imágenes y máscaras.


Cargando imágenes y máscaras: 20it [02:46,  8.34s/it]


In [250]:
# Leer el archivo CSV con las características y etiquetas
csv_file = "caracteristicas_imagenes_pixel_a_pixel.csv"
data = pd.read_csv(csv_file)

# Separar características (R, G, B, Sobel, HOG) y etiquetas
X = data[['R', 'G', 'B', 'sobel', 'hog']]  # Solo las características
y = data['etiqueta']  # Etiquetas (0 para fondo, 255 para carretera)

# Dividir los datos en entrenamiento (80%), validación (10%) y prueba (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Entrenar el modelo Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42, verbose=2, n_jobs=-1)
clf.fit(X_train, y_train)

# Evaluar en el conjunto de validación
y_val_pred = clf.predict(X_val)  # Predicción directa, ya no usamos umbral

# Evaluar en el conjunto de prueba
y_test_pred = clf.predict(X_test)  # Predicción directa

# Imprimir el informe de clasificación para el conjunto de prueba
print("Reporte de clasificación para el conjunto de prueba:")
print(classification_report(y_test, y_test_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   35.0s


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.6min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.9s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.4s


Reporte de clasificación para el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.61      0.62      0.62    170918
         255       0.61      0.60      0.60    166583

    accuracy                           0.61    337501
   macro avg       0.61      0.61      0.61    337501
weighted avg       0.61      0.61      0.61    337501



[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    2.0s finished
